In [ ]:
import sys
import os

notebook_path = os.getcwd() 
project_root = os.path.dirname(notebook_path)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

os.mkdir("models_sanity_check") if not os.path.exists("models_sanity_check") else None
model_storage_path = os.path.join(project_root, "testcases/models_sanity_check")

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [ ]:
import NN_models
import Filters
import utils
import Systems
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateKalmanNet import StateKalmanNet
from state_NN_models.StateKalmanNetWithKnownR import StateKalmanNetWithKnownR

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

In [ ]:
state_dim = 1
obs_dim = 1

# --- Reálný systém ("Ground Truth") ---
F_true = torch.tensor([[0.2]])
H_true = torch.tensor([[1.0]])
Q_true = torch.tensor([[2.5]])
R_true = torch.tensor([[0.1]])
Ex0_true = torch.tensor([[1.0]])  # Střední hodnota počátečního stavu
P0_true = torch.tensor([[1.5]])   # Počáteční kovariance

# --- Model systému ---
F_model = torch.tensor([[0.7]])
H_model = H_true
Q_model = torch.tensor([[0.1]]) 
R_model = R_true
Ex0_model = torch.tensor([[0.5]])  # Schválně jiná střední hodnota
P0_model = torch.tensor([[1.0]])   # Schválně jiná počáteční kovariance


sys_true = Systems.DynamicSystem(
    state_dim=state_dim,
    obs_dim=obs_dim,
    Ex0=Ex0_true,
    P0=P0_true,
    Q=Q_true,
    R=R_true,
    F=F_true,
    H=H_true,
    device=device
)

sys_model = Systems.DynamicSystem(
    state_dim=state_dim,
    obs_dim=obs_dim,
    Ex0=Ex0_model,
    P0=P0_model,
    Q=Q_model,
    R=R_model,
    F=F_model,
    H=H_model,
    device=device
)

print("\nInicializace systémů dokončena.")
print(f"Reálný systém: f(x) = {sys_true.F.item():.2f}*x, h(x) = {sys_true.H.item():.2f}*x")
print(f"Model systému: f(x) = {sys_model.F.item():.2f}*x, h(x) = {sys_model.H.item():.2f}*x")

In [ ]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 20      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 200      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 2000   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 500    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 16         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
kalman_filter = Filters.ExtendedKalmanFilter(sys_model)
y_first_trajectory = y_test[0] 
x_true_first_trajectory = x_test[0]

kf_results = kalman_filter.process_sequence(y_first_trajectory)

x_hat_kf = kf_results['x_filtered']
P_hat_kf = kf_results['P_filtered']

print("\nZpracování dokončeno.")
print(f"Tvar odhadnutých stavů: {x_hat_kf.shape}")
print(f"Tvar skutečných stavů:  {x_true_first_trajectory.shape}")

mse_kf = torch.nn.functional.mse_loss(x_hat_kf, x_true_first_trajectory)
print(f"MSE pro EKF na první trajektorii: {mse_kf.item():.4f}")

In [ ]:
adaptive_kalman_filter = Filters.AdaptiveExtendedKalmanFilter(sys_model,sys_model.Q,sys_model.R,alpha=0.99)
y_first_trajectory = y_test[0] 
x_true_first_trajectory = x_test[0]

adaptive_kf_results = adaptive_kalman_filter.process_sequence(y_first_trajectory)

x_hat_aekf = adaptive_kf_results['x_filtered']
P_hat_aekf = adaptive_kf_results['P_filtered']

print("\nZpracování dokončeno.")
print(f"Tvar odhadnutých stavů: {x_hat_aekf.shape}")
print(f"Tvar skutečných stavů:  {x_true_first_trajectory.shape}")

mse_aekf = torch.nn.functional.mse_loss(x_hat_aekf, x_true_first_trajectory)
print(f"MSE pro AEKF na první trajektorii: {mse_aekf.item():.4f}")



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Získání dat z výsledků filtru ---
x_hat_aekf = adaptive_kf_results['x_filtered']
P_hat_aekf = adaptive_kf_results['P_filtered']
K_aekf = adaptive_kf_results['Kalman_gain']

# Počet časových kroků
TEST_SEQ_LEN = x_hat_aekf.shape[0]
time = np.arange(TEST_SEQ_LEN)

# Převod tenzorů na NumPy pro plotování (a odstranění přebytečných dimenzí)
x_hat_aekf_numpy = x_hat_aekf.detach().cpu().numpy().squeeze()
x_true_numpy = x_true_first_trajectory.detach().cpu().numpy().squeeze()
P_hat_aekf_numpy = P_hat_aekf.detach().cpu().numpy().squeeze()
K_aekf_numpy = K_aekf.detach().cpu().numpy().squeeze()


# --- Vytvoření grafů ---
fig, ax = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
fig.suptitle('Vývoj klíčových veličin AEKF v čase', fontsize=16)

# 1. Graf: Odhadovaný a skutečný stav (x_filtered)
ax[0].plot(time, x_hat_aekf_numpy, label='Odhad AEKF (x_filtered)', color='blue')
ax[0].plot(time, x_true_numpy, label='Skutečný stav (x_true)', color='green', linestyle='--')
ax[0].set_title('Porovnání odhadovaného a skutečného stavu')
ax[0].set_ylabel('Hodnota stavu x')
ax[0].legend()
ax[0].grid(True)

# 2. Graf: Kovariance chyby (P_filtered)
ax[1].plot(time, P_hat_aekf_numpy, label='Kovariance chyby (P_filtered)', color='red')
ax[1].set_title('Vývoj kovarianční matice chyby P')
ax[1].set_ylabel('Hodnota P')
ax[1].grid(True)
# Pro lepší vizualizaci kolapsu můžete zkusit logaritmickou škálu:
# ax[1].set_yscale('log')

# 3. Graf: Kalmanův zisk (K)
ax[2].plot(time, K_aekf_numpy, label='Kalmanův zisk (K)', color='purple')
ax[2].set_title('Vývoj Kalmanova zisku K')
ax[2].set_ylabel('Hodnota K')
ax[2].set_xlabel('Časový krok')
ax[2].grid(True)

# Zobrazit grafy
plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.show()

In [ ]:
model_storage_name = "state_kalman_net.pth"
if os.path.exists(os.path.join(model_storage_path, model_storage_name)):
    print(f"Našel jsem uložený model {model_storage_name}, načítám jej...")
    state_knet = StateKalmanNet(sys_model, device=device, hidden_size_multiplier=10).to(device)
    state_knet.load_state_dict(torch.load(os.path.join(model_storage_path, model_storage_name), map_location=device))
    state_knet.eval()
    print("Model načten.")
else:
    print(f"Nenašel jsem uložený model {model_storage_name}, trénuji nový model...")
    state_knet = StateKalmanNet(sys_model, device=device, hidden_size_multiplier=10).to(device)
    utils.train_state_KalmanNet(
        model=state_knet, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        device=device, 
        epochs=150, 
        lr=1e-3,
        early_stopping_patience=15
    )
    utils.store_model(state_knet, os.path.join(model_storage_path, model_storage_name))

In [ ]:
model_storage_name = "state_kalman_net_with_known_R.pth"
if os.path.exists(os.path.join(model_storage_path, model_storage_name)):
    print(f"Našel jsem uložený model {model_storage_name}, načítám jej...")
    state_knet_R = StateKalmanNetWithKnownR(sys_model, device=device, hidden_size_multiplier=10).to(device)
    state_knet_R.load_state_dict(torch.load(os.path.join(model_storage_path, model_storage_name), map_location=device))
    state_knet_R.eval()
    print("Model načten.")
else:
    print(f"Nenašel jsem uložený model {model_storage_name}, trénuji nový model...")
    state_knet_R = StateKalmanNetWithKnownR(sys_model, device=device, hidden_size_multiplier=10).to(device)
    utils.train_state_KalmanNet(
        model=state_knet_R, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        device=device, 
        epochs=150, 
        lr=1e-3,
        early_stopping_patience=15
    )
    utils.store_model(state_knet_R, os.path.join(model_storage_path, model_storage_name))

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2


def calculate_anees(x_true, x_hat, P_hat):
    """
    Vypočítá Average NEES (ANEES) pro JEDNU trajektorii.
    """
    seq_len, state_dim = x_true.shape
    nees_samples = torch.zeros(seq_len, device=x_true.device)

    for t in range(seq_len):
        error = x_true[t] - x_hat[t]
        P_t = P_hat[t]
            
        P_t_matrix = P_t.reshape(state_dim, state_dim)
        P_stable = P_t_matrix + torch.eye(state_dim, device=P_t_matrix.device) * 1e-9
        
        try:
            P_inv = torch.inverse(P_stable)
            nees_samples[t] = error.unsqueeze(0) @ P_inv @ error.unsqueeze(-1)
        except torch.linalg.LinAlgError:
            nees_samples[t] = float('nan')
            
    return torch.nanmean(nees_samples).item()

TEST_SEQ_LEN = 200
NUM_TEST_TRAJ = 100

print(f"Generuji {NUM_TEST_TRAJ} testovacích trajektorií...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)

state_knet.eval()
state_knet_R.eval()

all_mse_knet, all_mse_knet_R, all_mse_ekf, all_mse_aekf = [], [], [], []
all_anees_knet_R, all_anees_ekf, all_anees_aekf = [], [],[]

print(f"Vyhodnocuji na {NUM_TEST_TRAJ} testovacích trajektoriích...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        y_test_seq = y_test[i].to(device)
        x_true_seq = x_test[i].to(device)
        
        initial_state_gpu = x_true_seq[0, :].unsqueeze(0)
        state_knet.reset(batch_size=1, initial_state=initial_state_gpu)
        knet_predictions_x_gpu = []
        for t in range(1, TEST_SEQ_LEN):
            y_t = y_test_seq[t, :].unsqueeze(0)
            x_filtered_t = state_knet.step(y_t)
            knet_predictions_x_gpu.append(x_filtered_t.squeeze(0))
        
        x_hat_knet_run_gpu = torch.stack(knet_predictions_x_gpu, dim=0)
        x_hat_knet_full = torch.cat([initial_state_gpu, x_hat_knet_run_gpu], dim=0).cpu()

        state_knet_R.reset(batch_size=1, initial_state=initial_state_gpu)
        knet_R_predictions_x_gpu, knet_R_predictions_P_gpu = [], []
        for t in range(1, TEST_SEQ_LEN):
            y_t = y_test_seq[t, :].unsqueeze(0)
            x_filtered_t, P_filtered_t = state_knet_R.step(y_t)
            knet_R_predictions_x_gpu.append(x_filtered_t.squeeze(0))
            knet_R_predictions_P_gpu.append(P_filtered_t.squeeze(0))
            
        x_hat_knet_R_run_gpu = torch.stack(knet_R_predictions_x_gpu, dim=0)
        P_hat_knet_R_run_gpu = torch.stack(knet_R_predictions_P_gpu, dim=0)
        
        x_hat_knet_R_full = torch.cat([initial_state_gpu, x_hat_knet_R_run_gpu], dim=0).cpu()
        P0_gpu = sys_model.P0.to(device)
        P_hat_knet_R_full = torch.cat([P0_gpu.unsqueeze(0), P_hat_knet_R_run_gpu], dim=0).cpu()
        
        ekf_instance = Filters.ExtendedKalmanFilter(sys_model)
        Ex0_run = x_true_seq[0, :].unsqueeze(-1)
        P0_run = sys_model.P0
        ekf_results = ekf_instance.process_sequence(
            y_test_seq, Ex0=Ex0_run, P0=P0_run
        )
        x_hat_ekf_full = ekf_results['x_filtered'].cpu()
        P_hat_ekf_full = ekf_results['P_filtered'].cpu()

        aekf_instance = Filters.AdaptiveExtendedKalmanFilter(sys_model, sys_model.Q, sys_model.R, alpha=0.99)
        Ex0_run = x_true_seq[0, :].unsqueeze(-1)
        P0_run = sys_model.P0
        aekf_results = aekf_instance.process_sequence(
            y_test_seq, Ex0=Ex0_run, P0=P0_run
        )
        x_hat_aekf_full = aekf_results['x_filtered'].cpu()
        P_hat_aekf_full = aekf_results['P_filtered'].cpu()

        x_true_seq_cpu = x_true_seq.cpu()
        state_dim = x_true_seq_cpu.shape[1]
        
        all_mse_knet.append(F.mse_loss(x_hat_knet_full, x_true_seq_cpu).item())
        all_mse_knet_R.append(F.mse_loss(x_hat_knet_R_full, x_true_seq_cpu).item())
        all_mse_ekf.append(F.mse_loss(x_hat_ekf_full, x_true_seq_cpu).item())
        all_mse_aekf.append(F.mse_loss(x_hat_aekf_full, x_true_seq_cpu).item())

        all_anees_knet_R.append(calculate_anees(x_true_seq_cpu, x_hat_knet_R_full, P_hat_knet_R_full))
        all_anees_ekf.append(calculate_anees(x_true_seq_cpu, x_hat_ekf_full, P_hat_ekf_full))
        all_anees_aekf.append(calculate_anees(x_true_seq_cpu, x_hat_aekf_full, P_hat_aekf_full))

avg_mse_knet = np.mean(all_mse_knet)
avg_mse_knet_R = np.mean(all_mse_knet_R)
avg_mse_ekf = np.mean(all_mse_ekf)
avg_mse_aekf = np.mean(all_mse_aekf)
avg_anees_knet_R = np.mean(all_anees_knet_R)
avg_anees_ekf = np.mean(all_anees_ekf)
avg_anees_aekf = np.mean(all_anees_aekf)

print("\n" + "="*50)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*50)
print("\n--- Průměrná MSE (přesnost) ---")
print(f"StateKalmanNet:          {avg_mse_knet:.4f}")
print(f"StateKalmanNetWithKnownR:{avg_mse_knet_R:.4f}")
print(f"EKF:                     {avg_mse_ekf:.4f}")
print(f"AEKF:                    {avg_mse_aekf:.4f}")
print("\n--- Průměrný ANEES (kredibilita/kalibrace) ---")
print(f"Očekávaná hodnota: {state_dim:.4f}")
print("---------------------------------------")
print(f"StateKalmanNetWithKnownR:{avg_anees_knet_R:.4f}")
print(f"EKF:                     {avg_anees_ekf:.4f}")
print(f"AEKF:                    {avg_anees_aekf:.4f}")
print("="*50)

plt.figure(figsize=(18, 9))
plt.title("Porovnání filtrů (poslední trajektorie)", fontsize=16)

x_true_plot = x_true_seq.cpu().numpy().squeeze() 

knet_mean_plot = x_hat_knet_full.numpy().squeeze()
knet_R_mean_plot = x_hat_knet_R_full.numpy().squeeze()
ekf_mean_plot = x_hat_ekf_full.numpy().squeeze()
aekf_mean_plot = x_hat_aekf_full.numpy().squeeze()
knet_R_std_plot = torch.sqrt(torch.diagonal(P_hat_knet_R_full, dim1=-2, dim2=-1)).numpy().squeeze()
time_steps = range(len(x_true_plot))

plt.plot(time_steps, x_true_plot, 'k-', linewidth=3, label="Skutečný stav")
plt.plot(time_steps, knet_mean_plot, 'b:', linewidth=2, label=f"KNet (MSE={all_mse_knet[-1]:.4f})")
plt.plot(time_steps, knet_R_mean_plot, 'c--', linewidth=2.5, label=f"KNet+R (MSE={all_mse_knet_R[-1]:.4f})")
plt.plot(time_steps, ekf_mean_plot, 'g-.', linewidth=2, label=f"EKF (MSE={all_mse_ekf[-1]:.4f})")
plt.plot(time_steps, aekf_mean_plot, 'm-', linewidth=2, label=f"AEKF (MSE={all_mse_aekf[-1]:.4f})")
plt.fill_between(time_steps, knet_R_mean_plot - 2 * knet_R_std_plot, knet_R_mean_plot + 2 * knet_R_std_plot, 
                 color='cyan', alpha=0.2, label='Nejistota KNet+R (95%)')

plt.xlabel("Časový krok (t)", fontsize=12)
plt.ylabel("Hodnota", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.show()